# CICIDS - Shallow Models (DT)

The CICIDS2017 dataset is a comprehensive dataset for network intrusion detection, created by the Canadian Institute for Cybersecurity. It includes a diverse set of attack scenarios and normal traffic, making it suitable for training and evaluating intrusion detection systems.

The dataset includes various types of attacks such as Brute Force, Heartbleed, Botnet, DoS (Denial of Service), DDoS (Distributed Denial of Service), Web attacks, and Infiltration of the network from inside.

In [1]:
model_name = "decision_trees_(DT)"

In [2]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

## Step 1. Read data and import necessary libraries

In [3]:
import pandas as pd
df_train = pd.read_csv("../data/concatenated/concat.csv")

In [4]:
df_train.head(5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,0,0,0,40,0,5.000000e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,6.666667e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [5]:
df_train.shape

(2830743, 79)

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [7]:
df_train.describe()

c:\Users\Harman\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\Harman\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.829385e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2830743.0,2.830743e+06,2830743.0,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2830743.0,2830743.0,2830743.0,2830743.0,2830743.0,2830743.0,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06
mean,8.071483e+03,1.478566e+07,9.361160e+00,1.039377e+01,5.493024e+02,1.616264e+04,2.075999e+02,1.871366e+01,5.820194e+01,6.891013e+01,8.708495e+02,4.104958e+01,3.059493e+02,3.353257e+02,inf,inf,1.298449e+06,2.919271e+06,9.182475e+06,1.623796e+05,1.448296e+07,2.610193e+06,3.266957e+06,9.042939e+06,1.021893e+06,9.893830e+06,1.805784e+06,1.485973e+06,4.684692e+06,9.672614e+05,4.644646e-02,0.0,1.112782e-04,0.0,-2.599739e+04,-2.273275e+03,6.386535e+04,6.995192e+03,1.643450e+01,9.504024e+02,1.719444e+02,2.949756e+02,4.861548e+05,3.537976e-02,4.644646e-02,2.423392e-04,2.980705e-01,3.158443e-01,9.482316e-02,1.112782e-04,2.433990e-04,6.835004e-01,1.919837e+02,5.820194e+01,3.059493e+02,-2.599739e+04,0.0,0.0,0.0,0.0,0.0,0.0,9.361160e+00,5.492919e+02,1.039377e+01,1.616230e+04,6.989837e+03,1.989433e+03,5.418218e+00,-2.741688e+03,8.155132e+04,4.113412e+04,1.531825e+05,5.829582e+04,8.316037e+06,5.038439e+05,8.695752e+06,7.920031e+06
std,1.828363e+04,3.365374e+07,7.496728e+02,9.973883e+02,9.993589e+03,2.263088e+06,7.171848e+02,6.033935e+01,1.860912e+02,2.811871e+02,1.946367e+03,6.886260e+01,6.052568e+02,8.396932e+02,NaN,NaN,4.507944e+06,8.045870e+06,2.445954e+07,2.950282e+06,3.357581e+07,9.525722e+06,9.639055e+06,2.452916e+07,8.591436e+06,2.873661e+07,8.887197e+06,6.278469e+06,1.716095e+07,8.308983e+06,2.104500e-01,0.0,1.054826e-02,0.0,2.105286e+07,1.452209e+06,2.475371e+05,3.815170e+04,2.523772e+01,2.028229e+03,3.054915e+02,6.318001e+02,1.647490e+06,1.847378e-01,2.104500e-01,1.556536e-02,4.574107e-01,4.648513e-01,2.929706e-01,1.054826e-02,1.559935e-02,6.804920e-01,3.318603e+02,1.860912e+02,6.052568e+02,2.105286e+07,0.0,0.0,0.0,0.0,0.0,0.0,7.496728e+02,9.980070e+03,9.973883e+02,2.26305

In [8]:
# Remove leading and trailing whitespaces from column names
df_train.columns = df_train.columns.str.strip()

In [9]:
df_train.columns

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Co

## Step 2. Data Cleaning

### A. Missing values

In [10]:
print(df_train.isna().sum().sum())

1358


In [11]:
df_train.dropna(subset=["Flow Bytes/s"], inplace=True)

In [12]:
print(df_train.isna().sum().sum())

0


### Inf. values

In [13]:
import numpy as np
df_train = df_train.replace([np.inf, -np.inf], np.nan).dropna()

## Step 3. Data Preparation

### A. Normalise numeric features

In [14]:
# Get all numerical columns
numerical_columns = df_train.select_dtypes(include="number").columns

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train[numerical_columns] = scaler.fit_transform(df_train[numerical_columns])

### B. Map Labels to Multi-class

In [16]:
df_train["Label"].value_counts()

Label
BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [17]:
attack_mapping = {
	"BENIGN": 0,
	"DoS Hulk": 1,
	"PortScan": 2,
	"DDoS": 3,
	"DoS GoldenEye": 4,
	"FTP-Patator": 5,
	"SSH-Patator": 6,
	"DoS slowloris": 7,
	"DoS Slowhttptest": 8,
	"Bot": 9,
	"Web Attack � Brute Force": 10,
	"Web Attack � XSS": 11,
	"Infiltration": 12,
	"Web Attack � Sql Injection": 13,
	"Heartbleed": 14,
}

df_train["Label"] = df_train["Label"].map(attack_mapping)

In [18]:
df_train["Label"].value_counts()

Label
0     2271320
1      230124
2      158804
3      128025
4       10293
5        7935
6        5897
7        5796
8        5499
9        1956
10       1507
11        652
12         36
13         21
14         11
Name: count, dtype: int64

### C. Data Splitting

In [19]:
X = df_train.drop(columns="Label")
y = df_train["Label"]

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### D. Apply SMOTE to balance the training data

In [21]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 1. Undersample the majority class
undersampling_strategy = {
    0: 1000,
    1: 1000,
    2: 1000,
    3: 1000,
    4: 1000,
    5: 1000,
    6: 1000,
    7: 1000,
    8: 1000,
    9: 1000,
	10: 1000,
}
rus = RandomUnderSampler(random_state=42, sampling_strategy=undersampling_strategy)
X_train_undersampled, y_train_undersampled = rus.fit_resample(X_train, y_train)

# 2. Oversample the minority class
smote = SMOTE(random_state=42, sampling_strategy="auto")
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_undersampled, y_train_undersampled)

In [22]:
# Check class distribution after SMOTE
from collections import Counter

print(f"Class distribution before SMOTE: {Counter(y_train)}")
print(f"Class distribution after SMOTE: {Counter(y_train_balanced)}")

Class distribution before SMOTE: Counter({0: 1817112, 1: 184342, 2: 126927, 3: 102239, 4: 8219, 5: 6363, 6: 4769, 7: 4630, 8: 4390, 9: 1515, 10: 1206, 11: 533, 12: 29, 13: 17, 14: 9})
Class distribution after SMOTE: Counter({0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000, 10: 1000, 11: 1000, 12: 1000, 13: 1000, 14: 1000})


## Step 4. Model

#### A. Defining the model

In [31]:
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

In [32]:
model = DecisionTreeClassifier(random_state=42, max_depth=10)

### B. Training the model

In [33]:
model.fit(X_train_balanced, y_train_balanced)

DecisionTreeClassifier(max_depth=10, random_state=42)

In [34]:
# Save the model
import joblib
joblib.dump(model, f"../models/shallow/{model_name}.joblib")


['../models/shallow/decision_trees_(DT).joblib']

### G. Evaluating the model

In [35]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
# y_pred = svm_model.predict(X_test)

# Balance the test set

# 1. Undersample the majority classes, and keep the values for the minority classes

SAMPLE_TARGET = 1000
undersampling_strategy_test_set = {
    0: SAMPLE_TARGET if y_test.value_counts().get(0) > SAMPLE_TARGET else y_test.value_counts().get(0),	# BENIGN
	1: SAMPLE_TARGET if y_test.value_counts().get(1) > SAMPLE_TARGET else y_test.value_counts().get(1),	# DoS Hulk
	2: SAMPLE_TARGET if y_test.value_counts().get(2) > SAMPLE_TARGET else y_test.value_counts().get(2),	# PortScan
	3: SAMPLE_TARGET if y_test.value_counts().get(3) > SAMPLE_TARGET else y_test.value_counts().get(3),	# DDoS
	4: SAMPLE_TARGET if y_test.value_counts().get(4) > SAMPLE_TARGET else y_test.value_counts().get(4),	# DoS GoldenEye
	5: SAMPLE_TARGET if y_test.value_counts().get(5) > SAMPLE_TARGET else y_test.value_counts().get(5),	# FTP-Patator
	6: SAMPLE_TARGET if y_test.value_counts().get(6) > SAMPLE_TARGET else y_test.value_counts().get(6),	# SSH-Patator
	7: SAMPLE_TARGET if y_test.value_counts().get(7) > SAMPLE_TARGET else y_test.value_counts().get(7),	# DoS slowloris
	8: SAMPLE_TARGET if y_test.value_counts().get(8) > SAMPLE_TARGET else y_test.value_counts().get(8),	# DoS Slowhttptest
	9: SAMPLE_TARGET if y_test.value_counts().get(9) > SAMPLE_TARGET else y_test.value_counts().get(9),	# Bot
	10: SAMPLE_TARGET if y_test.value_counts().get(10) > SAMPLE_TARGET else y_test.value_counts().get(10),	# Web Attack - Brute Force
	11: SAMPLE_TARGET if y_test.value_counts().get(11) > SAMPLE_TARGET else y_test.value_counts().get(11),	# Web Attack - XSS
	12: SAMPLE_TARGET if y_test.value_counts().get(12) > SAMPLE_TARGET else y_test.value_counts().get(12),	# Infiltration
	13: SAMPLE_TARGET if y_test.value_counts().get(13) > SAMPLE_TARGET else y_test.value_counts().get(13),	# Web Attack - SQL Injection
	14: SAMPLE_TARGET if y_test.value_counts().get(14) > SAMPLE_TARGET else y_test.value_counts().get(14),	# Heartbleed
}

rus_test = RandomUnderSampler(random_state=42, sampling_strategy=undersampling_strategy_test_set)
X_test_balanced, y_test_balanced = rus_test.fit_resample(X_test, y_test)


In [36]:
batch_size = 100  # Adjust based on your dataset size
y_pred = []

for i in tqdm(range(0, len(X_test_balanced), batch_size), desc="Predicting batches"):
    batch = X_test_balanced[i:i + batch_size]
    y_pred.extend(model.predict(batch))

y_pred = np.array(y_pred)

Predicting batches: 100%|██████████| 99/99 [00:00<00:00, 523.80it/s]


In [37]:
print(f"Accuracy: {accuracy_score(y_test_balanced, y_pred)}")
print(f"F1 Score: {f1_score(y_test_balanced, y_pred, average='weighted')}")

Accuracy: 0.952096414826818
F1 Score: 0.9539228941604226


In [38]:
print(classification_report(y_test_balanced, y_pred, target_names=attack_mapping.keys()))

                            precision    recall  f1-score   support

                    BENIGN       0.98      0.90      0.94      1000
                  DoS Hulk       0.97      0.96      0.97      1000
                  PortScan       0.99      0.98      0.99      1000
                      DDoS       1.00      1.00      1.00      1000
             DoS GoldenEye       0.95      0.98      0.97      1000
               FTP-Patator       1.00      1.00      1.00      1000
               SSH-Patator       1.00      1.00      1.00      1000
             DoS slowloris       0.93      0.99      0.96      1000
          DoS Slowhttptest       0.99      0.91      0.95      1000
                       Bot       0.95      1.00      0.97       441
  Web Attack � Brute Force       0.69      0.42      0.52       301
          Web Attack � XSS       0.34      0.81      0.48       119
              Infiltration       1.00      1.00      1.00         7
Web Attack � Sql Injection       0.15      0.75